You can download the `requirements.txt` for this course from the workspace of this lab. `File --> Open...`

# L2: Create Agents to Research and Write an Article

In this lesson, you will be introduced to the foundational concepts of multi-agent systems and get an overview of the crewAI framework.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [14]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import from the crewAI libray.

In [15]:
from crewai import Agent, Task, Crew

- As a LLM for your agents, you'll be using OpenAI's `gpt-3.5-turbo`.

**Optional Note:** crewAI also allow other popular models to be used as a LLM for your Agents. You can see some of the examples at the [bottom of the notebook](#1).

In [16]:
import os
from utils import get_openai_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'

## Creating Agents

- Define your Agents, and provide them a `role`, `goal` and `backstory`.
- It has been seen that LLMs perform better when they are role playing.

### Agent: Planner

**Note**: The benefit of using _multiple strings_ :
```Python
varname = "line 1 of text"
          "line 2 of text"
```

versus the _triple quote docstring_:
```Python
varname = """line 1 of text
             line 2 of text
          """
```
is that it can avoid adding those whitespaces and newline characters, making it better formatted to be passed to the LLM.

In [17]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    allow_delegation=False,
	verbose=True
)

### Agent: Writer

In [18]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True
)

### Agent: Editor

In [19]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True
)

## Creating Tasks

- Define your Tasks, and provide them a `description`, `expected_output` and `agent`.

### Task: Plan

In [20]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

### Task: Write

In [21]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

### Task: Edit

In [22]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

## Creating the Crew

- Create your crew of Agents
- Pass the tasks to be performed by those agents.
    - **Note**: *For this simple example*, the tasks will be performed sequentially (i.e they are dependent on each other), so the _order_ of the task in the list _matters_.
- `verbose=2` allows you to see all the logs of the execution. 

In [23]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=True
)

## Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [27]:
result = crew.kickoff(inputs={"topic": "Healthcare Providers in California, with particular focus on Kaiser Permanente, Sutter Health, UC Davis Health, and Dignity Health"})

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 0189c9ed-14d2-40da-b6e1-8c5793f05e7c                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: 9253f944-1d76-4c3c-b2b9-faf129f102eb
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: 9253f944-1d76-4c3c-b2b9-faf129f102eb
       Status: Executing Task...
    └── 🤖 Agent: Content Planner
            Status: In Progress

# Agent: Content Planner
## Task: 1. Prioritize the latest trends, key players, and noteworthy news on Healthcare Providers in California, with particular focus on Kaiser Permanente, Sutter Health, UC Davis Health, and Dignity Health.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


# Agent: Content Planner
## Final Answer: 
Content Plan Document:

Title: Healthcare Providers in California: A Comprehensive Overview of Kaiser Permanente, Sutter Health, UC Davis Health, and Dignity Health

Outline:
I. Introduction
    A. Brief overview of the healthcare landscape in California
    B. Introduction of key players: Kaiser Permanente, Sutter Health, UC Davis Health, and Dignity Health

II. Latest Trends in Healthcare Providers in California
    A. Telemedicine advancements
    B. Integrated care models
    C. F

🚀 Crew: crew
└── 📋 Task: 9253f944-1d76-4c3c-b2b9-faf129f102eb
       Status: Executing Task...
    └── 🤖 Agent: Content Planner
            Status: ✅ Completed

...

# Agent: Editor
## Task: Proofread the given blog post for grammatical errors and alignment with the brand's voice.


# Agent: Editor
## Final Answer: 
# Healthcare Providers in California: A Comprehensive Overview of Kaiser Permanente, Sutter Health, UC Davis Health, and Dignity Health

California's healthcare landscape boasts a wealth of healthcare providers working to serve the diverse needs of its population. Notable among these are Kaiser Permanente, Sutter Health, UC Davis Health, and Dignity Health. These key players stand out for their commitment to offering a broad spectrum of medical services and fostering innovation within the state.

The healthcare providers in California are actively embracing recent trends to elevate the quality of patient care. From leveraging telemedicine advancements for remote consultations to implementing integrated care models that promote holistic healthcare approaches, these institutions are at the forefront of adopting progressive practices. More

🚀 Crew: crew
├── 📋 Task: 9253f944-1d76-4c3c-b2b9-faf129f102eb
│      Assigned to: Content Planner
│      Status: ✅ Completed
│   └── 🤖 Agent: Content Planner
│           Status: ✅ Completed
├── 📋 Task: dbd4c2cc-c283-4a2d-aa3a-c0d6a964c066
│      Assigned to: Content Writer
│      Status: ✅ Completed
│   └── 🤖 Agent: Content Writer
│           Status: ✅ Completed
└── 📋 Task: ed269b2c-ef63-47cb-ac27-2748d535b00d
       Status: Executing Task...
    └── 🤖 Agent: Editor
            Status: ✅ Completed

🚀 Crew: crew
├── 📋 Task: 9253f944-1d76-4c3c-b2b9-faf129f102eb
│      Assigned to: Content Planner
│      Status: ✅ Completed
│   └── 🤖 Agent: Content Planner
│           Status: ✅ Completed
├── 📋 Task: dbd4c2cc-c283-4a2d-aa3a-c0d6a964c066
│      Assigned to: Content Writer
│      Status: ✅ Completed
│   └── 🤖 Agent: Content Writer
│           Status: ✅ Completed
└── 📋 Task: ed269b2c-ef63-47cb-ac27-2748d535b00d
       Assigned to: Editor
       Status: ✅ Completed
    └── 🤖 Agent: Editor
            Status: ✅ Completed

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: ed269b2c-ef63-47cb-ac27-2748d535b00d                                                                     │
│  Agent: Editor                                                                                                  │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 0189c9ed-14d2-40da-b6e1-8c5793f05e7c                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

- Display the results of your execution as markdown in the notebook.

In [28]:
# from IPython.display import Markdown
# Markdown(result)

from IPython.display import Markdown

# Extract the raw text or content from the CrewOutput object
text_result = str(result.raw) if hasattr(result, 'raw') else str(result)

# Display as Markdown
Markdown(text_result)

# Healthcare Providers in California: A Comprehensive Overview of Kaiser Permanente, Sutter Health, UC Davis Health, and Dignity Health

California's healthcare landscape boasts a wealth of healthcare providers working to serve the diverse needs of its population. Notable among these are Kaiser Permanente, Sutter Health, UC Davis Health, and Dignity Health. These key players stand out for their commitment to offering a broad spectrum of medical services and fostering innovation within the state.

The healthcare providers in California are actively embracing recent trends to elevate the quality of patient care. From leveraging telemedicine advancements for remote consultations to implementing integrated care models that promote holistic healthcare approaches, these institutions are at the forefront of adopting progressive practices. Moreover, a notable emphasis on preventive care and wellness programs underscores their dedication to proactive health management.

**Kaiser Permanente** boasts a storied history and is renowned for its integrated healthcare system. By providing services across various medical specialties, Kaiser Permanente has garnered high levels of patient satisfaction. With strategic facilities and locations strategically situated throughout California, Kaiser Permanente continues to champion patient-centric care and innovation.

**Sutter Health** operates an extensive network of healthcare facilities, delivering specialized services to diverse communities. Through impactful community engagement programs and strategic partnerships, Sutter Health showcases a steadfast commitment to enhancing healthcare accessibility and quality. Recent expansions and collaborations have further solidified Sutter Health's presence in the ever-evolving healthcare landscape.

**UC Davis Health**, recognized as an academic medical center, is a hub of research and innovation excellence. Collaboration with local communities ensures a patient-centered care ethos. UC Davis Health's relentless pursuit of advancing medical knowledge and providing top-tier patient care cements its reputation as a leading healthcare provider in California.

**Dignity Health** places a strong emphasis on compassionate care, reflecting in its remarkable achievements in patient outcomes. Boasting facilities that offer an extensive array of medical services while upholding a focus on sustainability and environmental stewardship, Dignity Health's commitment to ethical healthcare practices aligns seamlessly with the evolving expectations of the healthcare sector.

Residents of California seeking healthcare services, patients keen on exploring their options, healthcare professionals exploring career opportunities, and investors closely monitoring the healthcare market all stand to benefit from the varied services and strengths offered by these prominent healthcare providers. An informed understanding of each provider’s unique offerings can empower individuals to make decisions aligned with their healthcare needs.

I urge readers to explore the diverse healthcare options presented by Kaiser Permanente, Sutter Health, UC Davis Health, and Dignity Health. Prioritizing health awareness, regular check-ups, and informed healthcare decision-making can have a profound impact on individual well-being. For those wishing to delve deeper into the offerings of these healthcare providers, visiting their official websites or directly engaging with them can provide comprehensive insights for making enlightened decisions.

In conclusion, the healthcare providers in California, particularly Kaiser Permanente, Sutter Health, UC Davis Health, and Dignity Health, are continuously innovating and adapting to cater to the multifaceted healthcare requirements of the state's populace. Through a commitment to following the latest trends, fostering patient-centric care, and actively engaging with communities, these institutions play pivotal roles in shaping the future trajectory of healthcare in California.

Sources:
- [California Healthcare Foundation](https://www.chcf.org/)
- [California Department of Public Health](https://www.cdph.ca.gov/)
- Official websites of Kaiser Permanente, Sutter Health, UC Davis Health, and Dignity Health